Step 1: Init data reference into Vector Database (Qdrant)

In [ ]:
from model.AutoRouting import *
LOGGER = logger.Logger().get_logger(logger_type="embedding_")

qr = Qdrant()
model = AutoRouting()
model.clear_cuda_cache() 
current_hour = datetime.now().hour

In [2]:
# Máy tính bạn nào có Cuda thì mở comment

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# LOGGER.info("Using device:", device)

# Additional Info when using cuda
# if device.type == "cuda":
#     LOGGER.info(torch.cuda.get_device_name(0))
#     LOGGER.info("Memory Usage:")
#     LOGGER.info("Allocated: %.1f GB", round(torch.cuda.memory_allocated(0) / 1024**3, 1))
#     LOGGER.info("Cached:   %.1f GB", round(torch.cuda.memory_reserved(0) / 1024**3, 1))
# curdir = os.getcwd()

Step 2: Insert data reference to Qdrant vector database

In [ ]:
try:
    ref_database = pd.read_csv(os.path.join(os.getcwd(), 'data/data_reference.csv'))
    if len(ref_database) > 0: 
        df = await model.embedding(ref_database)
        df.to_pickle(os.path.join(os.getcwd(), 'data/data_reference.pkl'))
    else:
        LOGGER.info('Dont have any new hub')
except Exception as e:
    LOGGER.error(f"An error occurred while init new data_ref: {str(e)}")

Step 3: Main process: search similarity score

In [ ]:
hub_list = [1236]
thread_pool = ThreadPoolExecutor(max_workers=8)

async def _embedding_(hub_id, batch_size):
    try:
        LOGGER.info(f'Get data from BigQuery for hub {hub_id}')
        data = pd.read_csv(os.path.join(os.getcwd(), 'data/search.csv')).sort_values(by='hub_id')

        results_3 = pd.DataFrame()
        if len(data) > 0:
            results = await model.run(data, batch_size=batch_size)
            results_2 = pd.DataFrame(results)
            results_3 = data.merge(results_2, on='order_id', how='left')
            results_3.to_csv((os.path.join(os.getcwd(), 'data/result.csv')))
        del results_3
        
        LOGGER.info(f'Processed hub {hub_id} data with shape: {data.shape}')
            
    except Exception as e:
        LOGGER.error(f"Error processing hub {hub_id}: {str(e)}")

async def embedding(hub_list):
    LOGGER.info("Running search process")
    semaphore = asyncio.Semaphore(8) 
    async def semaphore_task(task):
        async with semaphore:
            await task
    tasks = [semaphore_task(_embedding_(hub_id, batch_size=1500)) for hub_id in hub_list]
    await asyncio.gather(*tasks)
    
async def main():
    try:
        start_time = datetime.now()
        await embedding(hub_list)
        time_taken = (datetime.now() - start_time).total_seconds() / 60
        LOGGER.info(f"Process completed in {time_taken:.2f} minutes")

    except Exception as e:
        LOGGER.error(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    asyncio.run(main())